In [ ]:
import numpy as np
import pickle

import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd

from typing import Union, List, Tuple

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020');

1.Znajdź listę wszystkich filmów o tej samej długości.

In [4]:
df = pd.read_sql("SELECT length,count(title) from film group by length",con = connection)
print(df)

     length  count
0       129      7
1       106      6
2       120      9
3       171      8
4       138      6
..      ...    ...
135     159      6
136      72      4
137     177      6
138     141      7
139     144      8

[140 rows x 2 columns]


2.Znajdź wszystkich klientów mieszkających w tym samym mieście.

In [8]:
df2 = pd.read_sql("SELECT city.city,count(customer_id) \
                  from ((customer inner join address on customer.address_id = address.address_id)\
                  inner join city on address.city_id=city.city_id)\
                  group by city",con = connection)
print(df2)

                     city  count
0                 Asuncin      1
1              Rae Bareli      1
2               Molodetno      1
3                 Yerevan      1
4    Vitria de Santo Anto      1
..                    ...    ...
592                  Abha      1
593               Kolpino      1
594          Garden Grove      1
595          Novoterkassk      1
596              Maracabo      1

[597 rows x 2 columns]


3.Oblicz średni koszt wypożyczenia wszystkich filmów.

In [10]:
df3 = pd.read_sql("SELECT avg(amount) from payment",con=connection)
print(df3)

        avg
0  4.200606


4.Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.

In [11]:
df4 = pd.read_sql("SELECT category.name,count(title) from ((film inner join film_category on film.film_id=film_category.film_id)\
inner join category on film_category.category_id = category.category_id)group by name",con=connection)
print(df4)

           name  count
0        Horror     56
1        Comedy     58
2        Sci-Fi     61
3         Drama     62
4       Foreign     73
5      Classics     57
6         Games     61
7           New     63
8        Travel     57
9         Music     51
10       Action     64
11  Documentary     68
12       Sports     74
13    Animation     66
14       Family     69
15     Children     60


5.Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.

In [17]:
df5 = pd.read_sql("SELECT country.country,count(customer_id) \
                  from ((customer inner join address on customer.address_id = address.address_id)\
                  inner join city on address.city_id=city.city_id)\
                  inner join country on city.country_id=country.country_id\
                  group by country",con = connection)
print(df5)

                  country  count
0                Cambodia      2
1                  Turkey     15
2                 Germany      7
3              Madagascar      1
4                    Chad      1
..                    ...    ...
103                 Sudan      2
104  United Arab Emirates      3
105              Tanzania      3
106      French Polynesia      2
107              Bulgaria      2

[108 rows x 2 columns]


6.Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.

In [28]:
df6 = pd.read_sql("Select store.address_id,store.store_id,count(customer_id)\
                  from customer inner join store on customer.store_id=store.store_id \
                  group by store.store_id having count(*) between 100 and 300",con = connection)
print(df6) # tu trzeba wrocic i to porpawic

   address_id  store_id  count
0           2         2    273


7.Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.

In [42]:
df7 = pd.read_sql("Select customer.first_name,customer.last_name,sum(film.length)\
                from customer inner join rental on customer.customer_id=rental.customer_id\
                inner join inventory on rental.inventory_id=inventory.inventory_id\
                inner join film on inventory.film_id=film.film_id group by customer.first_name,customer.last_name \
                having sum(film.length) > 1200",con=connection)
print(df7)

   first_name  last_name   sum
0         Tim       Cary  4476
1      Regina      Berry  4051
2     Jessica       Hall  4150
3      Marion     Snyder  4599
4       Daisy      Bates  4369
5       Tommy    Collazo  4051
6     Brandon       Huey  4160
7      Rhonda    Kennedy  4307
8     Eleanor       Hunt  5360
9      Janice       Ward  4366
10     Marsha    Douglas  4154
11     Sherry   Marshall  4582
12      Naomi   Jennings  4164
13     Warren    Sherrod  4233
14     Sherri     Rhodes  4030
15      Steve  Mackenzie  4206
16     Martha   Gonzalez  4237
17      Clara       Shaw  4808
18      Elsie     Kelley  4014
19     Justin        Ngo  4550
20     Angela  Hernandez  4362
21       Mike        Way  4023
22        Sue     Peters  4319
23   Michelle      Clark  4157
24  Elizabeth      Brown  4531
25      Grace      Ellis  4060
26     Wesley       Bull  4808
27      Louis      Leone  4333
28      Brett   Cornwell  4197
29       June    Carroll  4261
30       Karl       Seal  5388
31     C